In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [3]:
# 파일 로드
df_welfare = pd.read_csv('../welfare_school_concat_EPSG_5186.csv')
df_greenlight_caution = pd.read_csv('./서울시_음향신호기_관련정보.csv',encoding='euc-kr')

In [4]:
# 좌표 데이터를 Shapely Point 객체로 변환
gdf_welfare = gpd.GeoDataFrame(df_welfare, geometry=[Point(xy) for xy in zip(df_welfare['X'], df_welfare['Y'])])
gdf_greenlight_caution = gpd.GeoDataFrame(df_greenlight_caution, geometry=[Point(xy) for xy in zip(df_greenlight_caution['X좌표'], df_greenlight_caution['Y좌표'])])

In [5]:
# GeoDataFrame에 좌표계 설정
gdf_welfare.set_crs(epsg=5186, inplace=True)
gdf_greenlight_caution.set_crs(epsg=5186, inplace=True)

음향신호관리번호         지주관리번호  방향 (공통)       설치일       교체일  \
0      24-0000019810  02-0000220951      NaN  20221121  20221121   
1      24-0000010308  02-0000087079    180.0  20000101  20000101   
2      24-0000012524  02-0000081308      0.0       NaN       NaN   
3      24-0000009796  02-0000118897    180.0  20131207  20131207   
4      24-0000021536  02-0000180500      NaN  20230831  20230831   
...              ...            ...      ...       ...       ...   
26632  24-0000021859  02-0000223738      NaN  20231103  20231103   
26633  24-0000021858  02-0000223736      NaN  20231103  20231103   
26634  24-0000017646  02-0000082889      NaN  20201202  20201202   
26635  24-0000021857  02-0000090709      NaN  20231113  20231113   
26636  24-0000021856  02-0000090669      NaN  20231113  20231113   

                X좌표           Y좌표       제조회사 시설번호  작업구분 (공통)  표출구분 (공통)  종류  \
0      202410.75432  540397.01740  (주)한길에이치씨  NaN          1          2   1   
1      203867.87114  563343.20795        NaN  NaN          1          2   1   
2      207048.17500  553518.03124        NaN  NaN          1          2   1   
3      207771.50524  557685.44648  (주)한길에이치씨  NaN          1          2   1   
4               NaN           NaN       대한신호  NaN          1          2   1   
...             ...           ...        ...  ...        ...        ...  ..   
26632           NaN           NaN       대한신호  NaN          1          2   1   
26633           NaN           NaN       대한신호  NaN          1          2   1   
26634  204845.88245  558851.71294       대한신호  NaN          1          2   1   
26635           NaN           NaN       대한신호  NaN          1          2   1   
26636           NaN           NaN       대한신호  NaN          1          2   1   

           신규정규화ID  상태 (공통)         공사관리번호 음향신호관리번호.1   이력ID  위치정보 공사형태 (공통)  \
0      4057421          1.0  2022-0101-101  24-019810  24538   NaN       001   
1      4583314          1.0            NaN  24-010308  15036   1.0       001   
2      5343753          1.0            NaN  24-012524  17252   1.0       001   
3      5461184          1.0  2013-1801-007  24-009796  14524   1.0       001   
4      1270382          1.0  2023-0101-013  24-021536  26264   NaN       001   
...            ...      ...            ...        ...    ...   ...       ...   
26632  5404318          1.0  2023-0101-133  24-021859  26587   NaN       001   
26633  5404316          1.0  2023-0101-133  24-021858  26586   NaN       001   
26634  5404315          1.0  2023-0101-133  24-017646  22374   NaN       004   
26635  5513301          1.0  2023-0101-133  24-021857  26585   NaN       001   
26636  5513201          1.0  2023-0101-133  24-021856  26584   NaN       001   

                            geometry  
0      POINT (202410.754 540397.017)  
1      POINT (203867.871 563343.208)  
2      POINT (207048.175 553518.031)  
3      POINT (207771.505 557685.446)  
4                        POINT EMPTY  
...                              ...  
26632                    POINT EMPTY  
26633                    POINT EMPTY  
26634  POINT (204845.882 558851.713)  
26635                    POINT EMPTY  
26636                    POINT EMPTY  

[26637 rows x 20 columns]

In [6]:
# 복지시설의 각 위치에 대해 300미터 버퍼 생성
gdf_welfare['buffer'] = gdf_welfare.geometry.buffer(300)

In [7]:
# traffic equipment 개수를 계산하는 함수
def count_facilities_within_buffer(row, gdf_b):
    # 버퍼 내의 traffic equipment 필터링
    facilities_within_buffer = gdf_b[gdf_b.geometry.within(row.buffer)]
    return len(facilities_within_buffer)

In [8]:
# '안전표지 개수' 열 추가
gdf_welfare['음향신호기 개수'] = gdf_welfare.apply(lambda row: count_facilities_within_buffer(row, gdf_greenlight_caution), axis=1)

In [9]:
greenlight_caution_count = gdf_welfare[['시설명', '음향신호기 개수']].copy()
print(greenlight_caution_count)

              시설명  음향신호기 개수
0        송파구방이복지관         6
1      서울시각장애인복지관        12
2    서울시립발달장애인복지관         0
3           청음복지관        41
4      마포장애인종합복지관        12
..            ...       ...
675        서울정문학교        19
676        서울정진학교         0
677        한국우진학교        28
678         서울농학교        25
679         서울맹학교        25

[680 rows x 2 columns]
